<a href="https://colab.research.google.com/github/JakeOh/202511_BD53/blob/main/lab_python/da10_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터프레임 합치기

*   merge: 두 개 이상의 데이터프레임을 공통된 컬럼(들)을 기준으로 합치는 것. SQL의 join과 비슷.
*   concat: 두 개 이상의 데이터프레임을 축(axis)을 따라서 합치는 것.


# 모듈 임포트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# merge

In [2]:
emp = pd.DataFrame(data={
    'empno': [101, 102, 201, 202],
    'ename': ['Scott', 'King', 'Allen', '오쌤'],
    'deptno': [10, 20, 10, 40]
})
emp

,empno,ename,deptno
0,101,Scott,10
1,102,King,20
2,201,Allen,10
3,202,오쌤,40


In [3]:
dept = pd.DataFrame(data={
    'deptno': [10, 20, 30],
    'dname': ['HR', 'IT', 'Sales']
})
dept

,deptno,dname
0,10,HR
1,20,IT
2,30,Sales



*   `pd.merge(left, right, how, on, ...)` 함수
*   `pd.DataFrame.merge(right, how, on, ...)` 메서드
*   파라미터 `how`: merge 방식(inner, left, right, outer). 기본값은 inner.
*   파라미터 `on`: join 기준이 되는 컬럼(들의 리스트). join 컬럼 이름이 같은 경우에는 생략 가능.


## inner join

In [6]:
pd.merge(left=emp, right=dept)  # 함수(function)

,empno,ename,deptno,dname
0,101,Scott,10,HR
1,102,King,20,IT
2,201,Allen,10,HR


In [7]:
emp.merge(dept)  # 메서드(method)

,empno,ename,deptno,dname
0,101,Scott,10,HR
1,102,King,20,IT
2,201,Allen,10,HR


## left outer join

In [8]:
pd.merge(left=emp, right=dept, how='left')

,empno,ename,deptno,dname
0,101,Scott,10,HR
1,102,King,20,IT
2,201,Allen,10,HR
3,202,오쌤,40,NaN


In [9]:
emp.merge(dept, how='left')

,empno,ename,deptno,dname
0,101,Scott,10,HR
1,102,King,20,IT
2,201,Allen,10,HR
3,202,오쌤,40,NaN


## right outer join

In [10]:
pd.merge(left=emp, right=dept, how='right')

,empno,ename,deptno,dname
0,101.0,Scott,10,HR
1,201.0,Allen,10,HR
2,102.0,King,20,IT
3,NaN,NaN,30,Sales


In [11]:
emp.merge(dept, how='right')

,empno,ename,deptno,dname
0,101.0,Scott,10,HR
1,201.0,Allen,10,HR
2,102.0,King,20,IT
3,NaN,NaN,30,Sales


## full outer join

In [12]:
pd.merge(left=emp, right=dept, how='outer')

,empno,ename,deptno,dname
0,101.0,Scott,10,HR
1,201.0,Allen,10,HR
2,102.0,King,20,IT
3,NaN,NaN,30,Sales
4,202.0,오쌤,40,NaN


## join 조건 컬럼 이름이 다른 경우

In [13]:
dept2 = pd.DataFrame(data={
    'dno': [10, 20, 30],
    'dname': ['인사', '아이티', '영업']
})
dept2

,dno,dname
0,10,인사
1,20,아이티
2,30,영업


In [16]:
# inner join
emp.merge(right=dept2, left_on='deptno', right_on='dno')
#> left_on, right_on: 조인할 컬럼 이름.

,empno,ename,deptno,dno,dname
0,101,Scott,10,10,인사
1,102,King,20,20,아이티
2,201,Allen,10,10,인사


In [17]:
# left outer join
emp.merge(right=dept2, how='left', left_on='deptno', right_on='dno')

,empno,ename,deptno,dno,dname
0,101,Scott,10,10.0,인사
1,102,King,20,20.0,아이티
2,201,Allen,10,10.0,인사
3,202,오쌤,40,NaN,NaN


In [19]:
# right outer join
emp.merge(right=dept2, how='right', left_on='deptno', right_on='dno')

,empno,ename,deptno,dno,dname
0,101.0,Scott,10.0,10,인사
1,201.0,Allen,10.0,10,인사
2,102.0,King,20.0,20,아이티
3,NaN,NaN,NaN,30,영업


In [20]:
# full outer join
emp.merge(right=dept2, how='outer', left_on='deptno', right_on='dno')

,empno,ename,deptno,dno,dname
0,101.0,Scott,10.0,10.0,인사
1,201.0,Allen,10.0,10.0,인사
2,102.0,King,20.0,20.0,아이티
3,NaN,NaN,NaN,30.0,영업
4,202.0,오쌤,40.0,NaN,NaN


## 인덱스(row label)로 merge

In [25]:
np.random.seed(1)

In [26]:
df1 = pd.DataFrame(data={'value1': np.random.randint(10, size=6),
                         'value2': np.random.randint(100, size=6)},
                   index=['a', 'b', 'c'] * 2)
df1

,value1,value2
a,5,1
b,8,76
c,9,71
a,5,6
b,0,25
c,0,50


In [27]:
df2 = pd.DataFrame(data={'data1': np.random.rand(3),
                         'data2': np.random.rand(3)},
                   index=['a', 'b', 'c'])
df2

,data1,data2
a,0.846311,0.443453
b,0.313274,0.229577
c,0.524548,0.534414


In [28]:
pd.merge(left=df1, right=df2, left_index=True, right_index=True)
#> left_index와 right_index의 기본값은 False.
#> left_index 또는 right_index를 True로 설정하면, 조인할 때 인덱스를 사용함.

,value1,value2,data1,data2
a,5,1,0.846311,0.443453
b,8,76,0.313274,0.229577
c,9,71,0.524548,0.534414
a,5,6,0.846311,0.443453
b,0,25,0.313274,0.229577
c,0,50,0.524548,0.534414


In [29]:
df3 = pd.DataFrame(data={
    'value1': np.random.randint(10, size=6),
    'value2': np.random.randint(100, size=6),
    'value3': ['a', 'b', 'c'] * 2
})
df3

,value1,value2,value3
0,4,94,a
1,7,96,b
2,7,86,c
3,9,13,a
4,1,9,b
5,7,7,c


In [30]:
df4 = pd.DataFrame(data={'data1': [10, 20, 30]},
                   index=['a', 'b', 'c'])
df4

,data1
a,10
b,20
c,30


In [31]:
# df3 데이터프레임의 컬럼과 df4 데이터프레임의 인덱스를 사용해서 조인
pd.merge(left=df3, right=df4, left_on='value3', right_index=True)

,value1,value2,value3,data1
0,4,94,a,10
1,7,96,b,20
2,7,86,c,30
3,9,13,a,10
4,1,9,b,20
5,7,7,c,30


# concat

## Series에서의 concat

In [32]:
s1 = pd.Series(data=['scott', 'allen', 'king'])
s1

,0
0,scott
1,allen
2,king


In [33]:
s2 = pd.Series(data=['홍길동', '오쌤'])
s2

,0
0,홍길동
1,오쌤


In [34]:
pd.concat([s1, s2])  # objs 파라미터: Series 또는 DataFrame을 원소로 갖는 Iterable(list, tuple, ndarray)

,0
0,scott
1,allen
2,king
0,홍길동
1,오쌤


In [35]:
pd.concat([s1, s2], ignore_index=True)

,0
0,scott
1,allen
2,king
3,홍길동
4,오쌤


In [36]:
s3 = pd.Series(data=['HR', 'IT'])
s3

,0
0,HR
1,IT


In [38]:
pd.concat([s2, s3], axis=1)

,0,1
0,홍길동,HR
1,오쌤,IT


## DataFrame에서의 concat

In [41]:
np.random.seed(42)

In [42]:
score1 = pd.DataFrame(data={
    'id': np.arange(1, 6),
    'korean': np.random.randint(101, size=5),
    'english': np.random.randint(101, size=5),
    'math': np.random.randint(101, size=5)
})
score1

,id,korean,english,math
0,1,51,20,87
1,2,92,82,99
2,3,14,86,23
3,4,71,74,2
4,5,60,74,21


In [43]:
score2 = pd.DataFrame(data={
    'id': np.arange(6, 11),
    'korean': np.random.randint(101, size=5),
    'english': np.random.randint(101, size=5),
    'math': np.random.randint(101, size=5)
})
score2

,id,korean,english,math
0,6,52,1,75
1,7,1,63,57
2,8,87,59,21
3,9,29,20,88
4,10,37,32,48


In [45]:
pd.concat([score1, score2], ignore_index=True)  # axis=0 기본값

,id,korean,english,math
0,1,51,20,87
1,2,92,82,99
2,3,14,86,23
3,4,71,74,2
4,5,60,74,21
5,6,52,1,75
6,7,1,63,57
7,8,87,59,21
8,9,29,20,88
9,10,37,32,48


In [46]:
score3 = pd.DataFrame(data={
    'id': np.arange(1, 6),
    'science': np.random.randint(101, size=5),
    'history': np.random.randint(101, size=5),
    'art': np.random.randint(101, size=5)
})
score3

,id,science,history,art
0,1,90,79,61
1,2,58,14,50
2,3,41,61,54
3,4,91,61,63
4,5,59,46,2


In [47]:
pd.concat([score1, score3], axis=1)

,id,korean,english,math,id,science,history,art
0,1,51,20,87,1,90,79,61
1,2,92,82,99,2,58,14,50
2,3,14,86,23,3,41,61,54
3,4,71,74,2,4,91,61,63
4,5,60,74,21,5,59,46,2


In [49]:
pd.merge(left=score1, right=score3)

,id,korean,english,math,science,history,art
0,1,51,20,87,90,79,61
1,2,92,82,99,58,14,50
2,3,14,86,23,41,61,54
3,4,71,74,2,91,61,63
4,5,60,74,21,59,46,2


# merge, concat 연습 1.

미세먼지 농도 데이터셋

*   https://github.com/pandas-dev/pandas/tree/main/doc/data 데이터 사용
*   air_quality_no2_long.csv, air_quality_pm25_long.csv, air_quality_stations.csv 파일을 읽고, 각각 데이터프레임을 생성.
*   air_quality_no2 데이터프레임과 air_quality_pm25 데이터프레임을 합치세요.(merge? concat?)
*   위에서 만들어진 데이터프임과 air_quality_stations 데이터프레임을 합치세요.(merge? concat?)
*   데이터프레임 탐색


In [50]:
# 파일 경로
no2_file = 'https://github.com/pandas-dev/pandas/raw/refs/heads/main/doc/data/air_quality_no2_long.csv'
pm25_file = 'https://github.com/pandas-dev/pandas/raw/refs/heads/main/doc/data/air_quality_pm25_long.csv'
stations_file = 'https://github.com/pandas-dev/pandas/raw/refs/heads/main/doc/data/air_quality_stations.csv'